# Send a test event to an event hub
This will show how to send data to an event hub using a Neuroverse python library.This library has the following features:
- Sending small JSON serialisable object
- Sending data as references from an in memory bytes object
- Sending data as references from a locally accessible file
- Sending data as references from an Azure Blob
- References can be any data format as long as they can be read by python.
- References are automatically split into multiple messages if too large for one (We suggest preprocessing you references if larger than 400 MB to reduce their size)
- The wrapper allows for data of different forms to be colocated in the same partition reducing costs for low frequency data
- A helper has been written in pyspark to help read the landed data inside Neuroverse

This code would generally be run outside of Neuroverse closely located to the source of the data.

### Install the require libraries

In [ ]:
!pip install azure-eventhub==5.0.0 --upgrade
!pip install azure-storage==0.36.0 --upgrade


### Import required libraries

In [ ]:
import uuid
import json
import pickle
from neuro_python_clients.event_hub import chunked_sender as cs

### chunked_sender
The chunked_sender module has 2 import objects, a class called MessageDetails and a method called send_message that takes MessageDetails as a parameter.

### Construct and send message from in memory data

In [ ]:
referenceData = {"test":1,"test1":2}

interfaceName="testInterface1"
correlationId=str(uuid.uuid4())
version="v0.1"

data={"TestDetails":"InMemoryTest"}
connectionDetails=cs.InMemoryBytesConnectionDetails(bytes(json.dumps(referenceData),'utf-8'))

reference = cs.Reference("TestPayload1",cs.ReferenceType.InMemoryBytes,cs.ReferenceDataType.Json_UTF8,connectionDetails)
#use the connection string from 1. CreateEventHub
conn_str = "Endpoint=sb://.servicebus.windows.net/;SharedAccessKeyName=;SharedAccessKey=;EntityPath="
#partition ids range from 0 to 31
partition_id = "0"

msg_details=cs.MessageDetails(interfaceName,correlationId,version,data,reference,conn_str,partition_id)

cs.send_message_v1_4(msg_details)

### Construct and send message from local python pickle file

In [ ]:
#create file

referenceData = {"test":1,"test1":2}
with open('Test.pkl','wb') as file:
    file.write(pickle.dumps(referenceData))

interfaceName="testInterface2"
correlationId=str(uuid.uuid4())
version="v0.1"

data={"TestDetails":"LocalFileTest"}
connectionDetails=cs.LocalFileConnectionDetails('Test.pkl')

reference = cs.Reference("TestPayload2",cs.ReferenceType.LocalFile,cs.ReferenceDataType.Binary,connectionDetails)
#use the connection string from 1. CreateEventHub
conn_str = "Endpoint=sb://.servicebus.windows.net/;SharedAccessKeyName=;SharedAccessKey=;EntityPath="
#partition ids range from 0 to 31
partition_id = "0"

msg_details=cs.MessageDetails(interfaceName,correlationId,version,data,reference,conn_str,partition_id)

cs.send_message_v1_4(msg_details)

### Construct and send message from Azure blob

In [ ]:
interfaceName="testInterface3"
correlationId=str(uuid.uuid4())
version="v0.1"

data={"TestDetails":"AzureBlobTest"}

accountName=""
sasToken=""
containerName=""
blobName=""
connectionDetails=cs.AzureBlobConnectionDetails(accountName,sasToken,containerName,blobName)

reference = cs.Reference("TestPayload2",cs.ReferenceType.AzureBlobStore,cs.ReferenceDataType.Json_UTF8,connectionDetails)
#use the connection string from 1. CreateEventHub
conn_str = "Endpoint=sb://.servicebus.windows.net/;SharedAccessKeyName=;SharedAccessKey=;EntityPath="
#partition ids range from 0 to 31
partition_id = "0"

msg_details=cs.MessageDetails(interfaceName,correlationId,version,data,reference,conn_str,partition_id)

cs.send_message_v1_4(msg_details)